In [1]:
import networkx as nx
import numpy as np
from rdflib import Graph, URIRef
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
from rdflib.namespace import RDF, RDFS, SOSA

from viscars.data import DataLoader
from viscars.namespace import DASHB
from viscars.recommenders import Recommender
from viscars.utils import visualize_graph

In [2]:
class PPR(Recommender):

    def __init__(self, graph: Graph, verbose=False, alpha=0.8, tol=10e-6):
        super().__init__(graph, verbose)

        self.alpha = alpha
        self.tolerance = tol
        self.personalization = None

    def _build_model(self):
        graph_ = Graph()
        graph_ += self.graph.triples((None, None, None))

        self.items = list(graph_.subjects(RDF.type, SOSA.ObservableProperty))
        self.model = rdflib_to_networkx_multidigraph(graph_).to_undirected()

    def set_personalization(self, weight_uid=0, weight_cid=0):
        weight_others = 1 - weight_uid - weight_cid

        if weight_uid == 0:
            weight_uid = weight_others
        if weight_cid == 0:
            weight_cid = weight_others

        self.personalization = (weight_uid, weight_cid, weight_others)

    def _personalization(self, uid, cid):
        personalization = {}

        for node in self.model.nodes:
            uri = str(node)
            if uri in uid:
                personalization[node] = self.personalization[0]
            elif uri in cid:
                personalization[node] = self.personalization[1]
            else:
                personalization[node] = self.personalization[2]

        return personalization

    def run(self, uid: [] = None, cid: [] = None):
        if uid is not None and cid is not None:  # TODO: Check if UID and CID exist in graph -> ZeroDivisionError
            weights = self._personalization(uid, cid)
            return nx.pagerank(self.model, alpha=self.alpha, personalization=weights,
                               tol=self.tolerance)

        return nx.pagerank(self.model, alpha=self.alpha, tol=self.tolerance)

    def predict(self, uid: [] = None, cid: [] = None, **kwargs):
        pr = self.run(uid, cid)

        recommendations = \
            [{'contextId': cid, 'itemId': item, 'score': p} for item, p in pr.items()
             if item in self.items]
        return sorted(recommendations, key=lambda n: n['score'], reverse=True)

    def top_n(self, uid: [], cid: [], n: int, **kwargs):
        pass


In [3]:
graph = Graph()
graph.parse(f'./data/protego/protego_zplus.ttl', format='n3')
d_loader = DataLoader(graph)

In [4]:
recommender = PPR(graph)

In [5]:
patient = 'http://example.com/tx/patients/zplus_6'
properties = []

qry = f'''
    PREFIX dashb: <https://docs.dynamicdashboard.ilabt.imec.be/ontologies/dashboard#>
    PREFIX homelab: <https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/>
    PREFIX saref4ehaw: <https://saref.etsi.org/saref4ehaw/>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    PREFIX ssn: <http://www.w3.org/ns/ssn/>
    PREFIX ssn-ext: <http://localhost:8000/ontologies/ssn-extension/>
    
    SELECT ?property WHERE {{
       <{patient}> a saref4ehaw:HealthActor .
       
        ?sensor ssn-ext:subSystemOf <{patient}> ;
            sosa:observes ?property .
    }}
'''

result = graph.query(qry)
for row in result:
    properties.append(row.property)

In [6]:
recommendations = recommender.predict(uid=[URIRef(patient)])

filtered = []
for r in recommendations:
    if r['itemId'] in properties:
        filtered.append(r)

for r in filtered:
    item = str(r['itemId'])
    score = r['score']
    print(f'[{score}] {item}')

[0.00012387912260783523] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFE%253A44%253A6B%252Fservice0029/properties/org.dyamand.types.health.SpO2
[0.00012387912260783523] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.00%253A1C%253A05%253AFF%253AA9%253A4E%252Fservice0029/properties/org.dyamand.types.health.SpO2
[0.0001238705253518723] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.60%253A77%253A71%253A7D%253A93%253AD7%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel
[0.0001238705253518723] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.B0%253A91%253A22%253AFB%253AD0%253A78%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel
[0.00012375897423791958] https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.F8%253A33%253A31%253A46%253A6E%253A0B%252Fservice0010/properties/org.dyamand.types.health.BodyTemperature
[0.00012375897423791

In [7]:
diseases = []

qry = f'''
    PREFIX dashb: <https://docs.dynamicdashboard.ilabt.imec.be/ontologies/dashboard#>
    PREFIX homelab: <https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/>
    PREFIX saref4ehaw: <https://saref.etsi.org/saref4ehaw/>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    PREFIX ssn: <http://www.w3.org/ns/ssn/>
    PREFIX ssn-ext: <http://localhost:8000/ontologies/ssn-extension/>
    
    SELECT ?disease WHERE {{
       <{patient}> a saref4ehaw:HealthActor ;
           saref4ehaw:hasChronicDisease ?disease .
    }}
'''

result = graph.query(qry)

for row in result:
    diseases.append(row[0])
    
print(diseases)

[rdflib.term.URIRef('http://example.com/tx/diseases/Diabetes'), rdflib.term.URIRef('http://example.com/tx/diseases/Hypoglycemia')]


## Using PropertyClasses

In [8]:
patient = 'http://example.com/tx/patients/zplus_6'
properties_classes = []

qry = f'''
    PREFIX dashb: <https://docs.dynamicdashboard.ilabt.imec.be/ontologies/dashboard#>
    PREFIX homelab: <https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/>
    PREFIX saref4ehaw: <https://saref.etsi.org/saref4ehaw/>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    PREFIX ssn: <http://www.w3.org/ns/ssn/>
    PREFIX ssn-ext: <http://localhost:8000/ontologies/ssn-extension/>
    
    SELECT ?propertyClass WHERE {{
       <{patient}> a saref4ehaw:HealthActor .
       
        ?sensor ssn-ext:subSystemOf <{patient}> ;
            sosa:observes ?property .
        
        ?property a ?propertyClass .
        ?propertyClass a sosa:ObservableProperty .
    }}
'''

result = graph.query(qry)
for row in result:
    properties_classes.append(row.propertyClass)
    
print(properties_classes)

[rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.SpO2'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.SpO2'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.HeartRate'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.SystolicBloodPressure'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.GlucoseLevel'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.GlucoseLevel'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.DiastolicBloodPressure'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.BodyTemperature'), rdflib.term.URIRef('https://dahcc.idlab.ugent.be/Hom

In [9]:
recommendations = recommender.predict(uid=[URIRef(patient)])

filtered = []
for r in recommendations:
    if r['itemId'] in properties_classes:
        filtered.append(r)

for r in filtered:
    item = str(r['itemId'])
    score = r['score']
    print(f'[{score}] {item}')

[0.006879678394632745] https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.SpO2
[0.006876259676209813] https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.GlucoseLevel
[0.006804291825378031] https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.BodyTemperature
[0.003280178973683055] https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.DiastolicBloodPressure
[0.0032801789736830542] https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.SystolicBloodPressure
[0.0032213942973431204] https://dahcc.idlab.ugent.be/Homelab/SensorsAndWearables/org.dyamand.types.health.HeartRate


## Removing Sensor Metadata